In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from pyscf import fci, gto, scf, cc, mcscf

np.set_printoptions(precision=5, suppress=True)

In [ ]:
r = 1.6
nH = 10
# geom = f"H {-3*r/2} 0 0; H {-r/2} 0 0; H {r/2} 0 0; H {3*r/2} 0 0"
geom = ""
for i in range(nH):
    geom += "H 0 0 %g\n" % (i * r)
mol = gto.M(atom=geom, basis="sto-6g", verbose=3, unit="bohr")

mf = scf.RHF(mol)
mf.kernel()

umf = scf.UHF(mol)
umf.kernel()
mo1 = umf.stability(external=True)[0]
umf = umf.newton().run(mo1, umf.mo_occ)
mo1 = umf.stability(external=True)[0]
umf = umf.newton().run(mo1, umf.mo_occ)

# fci
cisolver = fci.FCI(mol, mf.mo_coeff)
e, ci = cisolver.kernel()
print("FCI energy: ", e)

### Free projection AFQMC

Evaluates the quantity

$$ E(\tau) = \frac{\langle \Psi_l | H e^{-\tau H} | \Psi_r \rangle}{\langle \Psi_l | e^{-\tau H} | \Psi_r \rangle} $$

where $|\Psi_l\rangle$ is a trial wave function and $|\Psi_r\rangle$ is an initial state. The propagator is sampled using Monte Carlo. $E(\tau)$ converges to the ground state energy at long $\tau$, but the energies get noisier at long $\tau$ due to the sign problem.

In the following, energy evaluations are performed after a block consisting of `num_steps` steps of duration `dt`. In one iteration, energy samples are collected at `num_blocks` different $\tau$ values. Multiple walkers are used to batch operations together for computational efficiency. The total number of samples at a given $\tau$ is given by `num_walkers` $\times$ `num_iterations_fp`. The energy is then averaged over walkers and iterations.


In [ ]:
from ipie.addons.free_projection.qmc.calc import build_fpafqmc_driver
from ipie.config import MPI
from ipie.utils.from_pyscf import gen_ipie_input_from_pyscf_chk

comm = MPI.COMM_WORLD

gen_ipie_input_from_pyscf_chk(umf.chkfile, verbose=0)
qmc_options = {
    "num_iterations_fp": 100,
    "num_blocks": 4,
    "num_steps": 30,
    "num_walkers": 50,
    "dt": 0.05,
}
afqmc = build_fpafqmc_driver(
    comm,
    nelec=mol.nelec,
    seed=212503,
    qmc_options=qmc_options,
)
afqmc.run()

# analysis
from ipie.addons.free_projection.analysis.extraction import extract_observable
from ipie.addons.free_projection.analysis.jackknife import jackknife_ratios

for i in range(afqmc.params.num_blocks):
    print(
        f"\nEnergy statistics at time {(i+1) * afqmc.params.num_steps_per_block * afqmc.params.timestep}:"
    )
    qmc_data = extract_observable(afqmc.estimators[i].filename, "energy")
    mean_energy, energy_err = jackknife_ratios(qmc_data["ENumer"], qmc_data["EDenom"])
    print(f"  Energy: {mean_energy:.8e} +- {energy_err:.8e}")

### Phaseless AFQMC


In [ ]:
from ipie.qmc.calc import build_afqmc_driver
from ipie.config import MPI
from ipie.utils.from_pyscf import gen_ipie_input_from_pyscf_chk

comm = MPI.COMM_WORLD

gen_ipie_input_from_pyscf_chk(mf.chkfile, verbose=0)

# fixing random seed for reproducibility
afqmc = build_afqmc_driver(comm, nelec=mol.nelec, num_walkers_per_task=100, seed=41100801)
if comm.rank == 0:
    print(afqmc.params)  # Inspect the default qmc options

# Let us override the number of blocks to keep it short
afqmc.params.num_blocks = 400
afqmc.run()

if comm.rank == 0:
    # We can extract the qmc data as as a pandas data frame like so
    from ipie.analysis.extraction import extract_observable

    qmc_data = extract_observable(afqmc.estimators.filename, "energy")
    y = qmc_data["ETotal"]
    y = y[50:]  # discard first 50 blocks

    from ipie.analysis.autocorr import reblock_by_autocorr

    df = reblock_by_autocorr(y, verbose=1)
    print(df.to_csv(index=False))
    # assert np.isclose(df.at[0,'ETotal_ac'], -5.325611614468466)
    # assert np.isclose(df.at[0,'ETotal_error_ac'], 0.00938082351500978)

### Multi-Slater trials and CCSD initial states in free projection


In [ ]:
r = 1.6
nH = 6
geom = ""
for i in range(nH):
    geom += "H 0 0 %g\n" % (i * r)
mol = gto.M(atom=geom, basis="6-31g", verbose=3, unit="bohr")

mf = scf.RHF(mol)
mf.kernel()

mycc = cc.CCSD(mf)
_ = mycc.kernel()

mc = mcscf.CASSCF(mf, nH, nH)
e_tot, e_cas, fcivec, mo, mo_energy = mc.kernel()
coeff, occa, occb = zip(
    *fci.addons.large_ci(fcivec, mc.ncas, mc.nelecas, tol=1e-8, return_strs=False)
)

# fci
cisolver = fci.FCI(mol, mf.mo_coeff)
e, ci = cisolver.kernel()
print("FCI energy: ", e)

In [ ]:
from ipie.addons.free_projection.propagation.free_propagation import FreePropagation
from ipie.addons.free_projection.propagation.CCSD import CCSD
from ipie.addons.free_projection.qmc.fp_afqmc import FPAFQMC
from ipie.addons.free_projection.qmc.options import QMCParamsFP
from ipie.addons.free_projection.walkers.uhf_walkers import UHFWalkersParticleHoleFP
from ipie.systems.generic import Generic
from ipie.trial_wavefunction.particle_hole import ParticleHole
from ipie.utils.from_pyscf import generate_hamiltonian
from ipie.walkers.walkers_dispatch import get_initial_walker
from ipie.utils.mpi import MPIHandler

In [ ]:
system = Generic(mc.nelecas)
ham = generate_hamiltonian(mol, mc.mo_coeff, mc.get_hcore(), mc.mo_coeff)

wave_function = (coeff, occa, occb)
trial = ParticleHole(wave_function, mc.nelecas, mol.nao, num_dets_for_trial=10)
trial.build()
trial.half_rotate(ham)

params = QMCParamsFP(
    num_walkers=50,
    total_num_walkers=50,
    num_blocks=4,
    num_steps_per_block=30,
    timestep=0.05,
    num_iterations_fp=100,
    rng_seed=212503,
)

_, initial_walker = get_initial_walker(trial)
walkers = UHFWalkersParticleHoleFP(
    initial_walker, mc.nelecas[0], mc.nelecas[1], ham.nbasis, params.num_walkers, None
)
walkers.build(trial)

ene_0 = mf.energy_tot()
propagator = FreePropagation(time_step=params.timestep, exp_nmax=10, ene_0=ene_0)
propagator.build(ham, trial, walkers, None)

orbital_rotation = mc.mo_coeff.T @ mf.get_ovlp() @ mf.mo_coeff
ccsd = CCSD(mycc.t1, mycc.t2, orbital_rotation)

mpi_handler = MPIHandler()

afqmc = FPAFQMC(system, ham, trial, walkers, propagator, mpi_handler, params, ccsd=ccsd)

afqmc.run()

# analysis
from ipie.addons.free_projection.analysis.extraction import extract_observable
from ipie.addons.free_projection.analysis.jackknife import jackknife_ratios

for i in range(afqmc.params.num_blocks):
    print(
        f"\nEnergy statistics at time {(i+1) * afqmc.params.num_steps_per_block * afqmc.params.timestep}:"
    )
    qmc_data = extract_observable(afqmc.estimators[i].filename, "energy")
    mean_energy, energy_err = jackknife_ratios(qmc_data["ENumer"], qmc_data["EDenom"])
    print(f"  Energy: {mean_energy:.8e} +- {energy_err:.8e}")